In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("player77/kryptonit-train")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/kryptonit-train


Разбиваю на триплеты для триплетного лоса: пара однорго человека + изображение другого  

In [2]:
import random
import json
from itertools import combinations

meta_path = "/kaggle/input/kryptonit-train/train/meta.json"



with open(meta_path, "r") as f:
    meta_data = json.load(f)


real_images = {}  #словарь для реальных изображений
fake_images = {}  #словарь для синтетических изображений

for img_path, label in meta_data.items():
    person_id, img_name = img_path.split("/")  #разделяем индекс человека и номер изображения

    if label == 0:
        if person_id not in real_images:
            real_images[person_id] = []
        real_images[person_id].append(img_path)
    else:
        if person_id not in fake_images:
            fake_images[f'{person_id}_fake'] = []
        fake_images[f'{person_id}_fake'].append(img_path)


from itertools import combinations
import random

positive_pairs = []
negative_pairs = []

#создаем пары
for person, images in real_images.items():
    if len(images) > 1:
        positive_pairs += [(i,1) for i in combinations(images, 2)] # Все возможные пары для одного человека
# пары без повторений и учета порядка - уникальные
# Создаем негативные пары
people = list(real_images.keys())
people_fake = list(fake_images.keys())
for _ in range(len(positive_pairs)):
    if random.choice([True, False]):
        p1, p2 = random.sample(people, 2)
        img1 = random.choice(real_images[p1])
        img2 = random.choice(real_images[p2])
        negative_pairs.append(((img1, img2), -1))
    else:
        p1 = random.choice(people)
        img1 = random.choice(real_images[p1])
        img2 = random.choice(fake_images[f'{p1}_fake'])
        negative_pairs.append(((img1, img2), -1))

print(f"✔ Создано {len(positive_pairs)} позитивных пар и {len(negative_pairs)} негативных пар.")
all_pairs = positive_pairs[:8000] + negative_pairs[:12000]

all_paits = random.shuffle(all_pairs)

✔ Создано 91552 позитивных пар и 91552 негативных пар.


In [ ]:
train_triplets[0]

Изучив TripletMarginLoss, мы увидели, что там используется евклидово расстояние, но по условию соревнования нам нужно оптимизировать косинусное. Поэтому перепишем функцию для косинусного расстояния:

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TripletLossWithCosineDistance(nn.Module):
    def __init__(self, margin=1):
        super(TripletLossWithCosineDistance, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        cosine_sim_pos = F.cosine_similarity(anchor, positive, dim=1)
        cosine_sim_neg = F.cosine_similarity(anchor, negative, dim=1)
        cosine_distance_pos = 1 - cosine_sim_pos
        cosine_distance_neg = 1 - cosine_sim_neg

        losses = torch.relu(cosine_distance_pos - cosine_distance_neg + self.margin)
        return losses.mean()

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader


class FaceEshkereDataset(Dataset):
  def __init__(self, pairs, mode):
    self.pairs = pairs
    self.mode = mode
    self.transform_train = transforms.Compose([
      transforms.Resize((112, 112)),
      transforms.RandomHorizontalFlip(p=0.2),  # Горизонтальный флип
      #transforms.ColorJitter(brightness=0.5, contrast=0.5),  # Изменение яркости и контраста
      transforms.ToTensor(),
      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
      ])
    self.transform_test = transforms.Compose([
      transforms.Resize((112, 112)),
      transforms.ToTensor(),
      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
      ])
  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, index):
    path_to_image0 = Path(f"/kaggle/input/kryptonit-train/train/images/{self.pairs[index][0][0]}") #надо бы поменять конечно, но работает
    path_to_image1 = Path(f"/kaggle/input/kryptonit-train/train/images/{self.pairs[index][0][1]}") #надо бы поменять конечно, но работает

    label = self.pairs[index][1]

    if self.mode == 'train':
      image2 = self.transform_train(Image.open(path_to_image0))
      image3 = self.transform_train(Image.open(path_to_image1))
    if self.mode == 'test':
      image2 = self.transform_test(Image.open(path_to_image0))
      image3 = self.transform_test(Image.open(path_to_image1))

    return image2, image3, label


In [4]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
#mport insightface
#from insightface.app import FaceAnalysis
import cv2
import numpy as np
#import insightface

import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import traceback
#import onnxruntime as ort
#from insightface.app import FaceAnalysis
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm
from PIL import Image

In [6]:
from sklearn.model_selection import train_test_split

train_pairs, val_pairs = train_test_split(all_pairs, test_size=0.1, random_state=42) #разобьем выборку на треин и валидацию

In [7]:
from torchvision import transforms


train_dataset = FaceEshkereDataset(train_pairs, 'train')
val_dataset = FaceEshkereDataset(val_pairs, 'test')

#test_dataset = FaceEshkereDataset(test_pairs, 'test') # для оценки метрики

#test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

In [8]:
!pip install huggingface_hub

from huggingface_hub import login

In [ ]:

# Ваш Access Token
token = "YOUR TOKEN"

# Авторизация
login(token)

Модель ArcFace

In [ ]:
import timm
import torch.nn.functional as F

model = timm.create_model("hf_hub:gaunernst/vit_tiny_patch8_112.arcface_ms1mv3", pretrained=True)

In [ ]:

weights = torch.load('/kaggle/input/weights/model_weights_epoch_20.pth', weights_only=True)
model.load_state_dict(weights)

Модель AdaFace

In [9]:
!pip install omegaconf
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=03e7b067b43e701fb5cca6690feed85e03599919b27f611b594af780664ad232
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=1d330272926192ef25a01b1ad5b9d895f42bd34316b2c63af46cc573bb8c86b6
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [10]:
from transformers import AutoModel
from huggingface_hub import hf_hub_download
import shutil
import os
import torch
import sys
from torchvision import transforms  

from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image

# helpfer function to download huggingface repo and use model
def download(repo_id, path, HF_TOKEN=None):
    os.makedirs(path, exist_ok=True)
    files_path = os.path.join(path, 'files.txt')
    if not os.path.exists(files_path):
        hf_hub_download(repo_id, 'files.txt', token=HF_TOKEN, local_dir=path, local_dir_use_symlinks=False)
    with open(os.path.join(path, 'files.txt'), 'r') as f:
        files = f.read().split('\n')
    for file in [f for f in files if f] + ['config.json', 'wrapper.py', 'model.safetensors']:
        full_path = os.path.join(path, file)
        if not os.path.exists(full_path):
            hf_hub_download(repo_id, file, token=HF_TOKEN, local_dir=path, local_dir_use_symlinks=False)

            
# helpfer function to download huggingface repo and use model
def load_model_from_local_path(path, HF_TOKEN=None):
    cwd = os.getcwd()
    os.chdir(path)
    sys.path.insert(0, path)
    model = AutoModel.from_pretrained(path, trust_remote_code=True, token=HF_TOKEN)
    os.chdir(cwd)
    sys.path.pop(0)
    return model


# helpfer function to download huggingface repo and use model
def load_model_by_repo_id(repo_id, save_path, HF_TOKEN=None, force_download=False):
    if force_download:
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
    download(repo_id, save_path, HF_TOKEN)
    return load_model_from_local_path(save_path, HF_TOKEN)



    
HF_TOKEN = 'YOUR_TOKEN'
path = os.path.expanduser('~/.cvlface_cache/minchul/cvlface_adaface_vit_base_webface4m')
repo_id = 'minchul/cvlface_adaface_vit_base_webface4m'
model = load_model_by_repo_id(repo_id, path, HF_TOKEN)

    # input is a rgb image normalized.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


files.txt:   0%|          | 0.00/296 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.pt:   0%|          | 0.00/460M [00:00<?, ?B/s]

model.yaml:   0%|          | 0.00/161 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/460M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loaded ViT model


/root/.cvlface_cache/minchul/cvlface_adaface_vit_base_webface4m/models/base/utils.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_loca

compatible keys in state_dict 281 / 281
Check


<All keys matched successfully>
Loaded pretrained model from pretrained_model/model.pt


In [13]:

for param in model.parameters():
    param.requires_grad = False

for layer in model.model.net.feature:
    if isinstance(layer, torch.nn.BatchNorm1d):
        for param in layer.parameters():
            param.requires_grad = False  # Замораживаем параметры

# Размораживаем Linear
for layer in model.model.net.feature:
    if isinstance(layer, torch.nn.Linear):
        for param in layer.parameters():
            param.requires_grad = True  



for name, param in model.named_parameters():
    print(f"Параметр {name} обучаемый: {param.requires_grad}")

Параметр model.net.pos_embed обучаемый: False
Параметр model.net.patch_embed.proj.weight обучаемый: False
Параметр model.net.patch_embed.proj.bias обучаемый: False
Параметр model.net.blocks.0.norm1.weight обучаемый: False
Параметр model.net.blocks.0.norm1.bias обучаемый: False
Параметр model.net.blocks.0.norm2.weight обучаемый: False
Параметр model.net.blocks.0.norm2.bias обучаемый: False
Параметр model.net.blocks.0.attn.qkv.weight обучаемый: False
Параметр model.net.blocks.0.attn.proj.weight обучаемый: False
Параметр model.net.blocks.0.attn.proj.bias обучаемый: False
Параметр model.net.blocks.0.mlp.fc1.weight обучаемый: False
Параметр model.net.blocks.0.mlp.fc1.bias обучаемый: False
Параметр model.net.blocks.0.mlp.fc2.weight обучаемый: False
Параметр model.net.blocks.0.mlp.fc2.bias обучаемый: False
Параметр model.net.blocks.1.norm1.weight обучаемый: False
Параметр model.net.blocks.1.norm1.bias обучаемый: False
Параметр model.net.blocks.1.norm2.weight обучаемый: False
Параметр model.ne

In [77]:
model

CVLFaceRecognitionModel(
  (model): ViTModel(
    (net): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 512, kernel_size=(8, 8), stride=(8, 8))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0): Block(
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=512, out_features=1536, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=512, out_features=512, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
          (mlp): Mlp(
            (fc1): Linear(in_features=512, out_features=1536, bias=True)
            (act): ReLU6()
            (fc2): Linear(in_features=1536, out_features=512, bias=True)
            (drop): Dropout(p=0.0, inplace=False

Обучение

In [14]:
print(f"Model device: {next(model.parameters()).device}")

Model device: cpu


In [15]:
import torch
from tqdm import tqdm
from torch.nn import DataParallel

def train(model, optimizer, criterion, epochs, train_dataloader, test_dataloader):
    summary_loss_train = []
    summary_loss_test = []
    
    # Определяем устройство (GPU или CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Перемещаем модель на GPU перед обертыванием в DataParallel
    model.to(device)
    
    # Обертываем модель в DataParallel для использования нескольких GPU
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = DataParallel(model)
    
    for epoch in tqdm(range(epochs), desc="Epochs", ncols=100):
        train_loss = 0
        train_norm_variable = 0
        model.train()
        
        for image0, image1, label in train_dataloader:
            # Перемещаем данные на GPU
            image0 = image0.to(device)
            image1 = image1.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            
            embed0 = model(image0)
            embed1 = model(image1)
            
            loss = criterion(embed0, embed1, label)
            summary_loss_train.append(loss.item())
            train_loss += loss.item() * image0.size(0)
            train_norm_variable += image0.size(0)
            
            loss.backward()
            optimizer.step()
        
        print(f'Train loss: {train_loss/train_norm_variable}')
        
        # Сохраняем веса модели
        torch.save(model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict(), 
                   f'/kaggle/working/model_weights_epoch_{epoch + 1}.pth')
        print(f"Weights saved for epoch {epoch + 1}")
        
        test_loss = 0
        test_norm_variable = 0
        model.eval()
        
        with torch.no_grad():
            for image0, image1, label in test_dataloader:
                # Перемещаем данные на GPU
                image0 = image0.to(device)
                image1 = image1.to(device)
                label = label.to(device)
                
                embed0 = model(image0)
                embed1 = model(image1)
                
                loss = criterion(embed0, embed1, label)
                summary_loss_test.append(loss.item())
                
                test_loss += loss.item() * image0.size(0)
                test_norm_variable += image0.size(0)
        
        print(f'Test loss: {test_loss/test_norm_variable}')
        print()

In [16]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=3e-3)
criterion = nn.CosineEmbeddingLoss()


In [17]:
train(model, optimizer, criterion, 15, train_dataloader, val_dataloader)

Using 2 GPUs!


Epochs:   0%|                                                                | 0/15 [00:00<?, ?it/s]

Train loss: 0.21841484558582305
Weights saved for epoch 1


Epochs:   7%|███▌                                                 | 1/15 [04:47<1:07:08, 287.73s/it]

Test loss: 0.13680170583724977

Train loss: 0.12443526046805911
Weights saved for epoch 2


...

Test loss: 0.07508446717262268

Train loss: 0.039333913491831886
Weights saved for epoch 11


Epochs:  73%|███████████████████████████████████████▌              | 11/15 [54:00<19:41, 295.25s/it]

Test loss: 0.07525718611478806



Epochs:  73%|███████████████████████████████████████▌              | 11/15 [54:12<19:42, 295.65s/it]


KeyboardInterrupt: 

In [24]:
train_all_dataset = FaceEshkereDataset(all_pairs, 'train')
train_all_dataloader = DataLoader(train_all_dataset, batch_size = 64, shuffle=True, num_workers=2, pin_memory=True)


In [27]:
def train_all(model, train_all_dataloader, criterion, optimizer, epochs):
    sum_loss =  0
    loss_norm = 0

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = DataParallel(model)
    for epoch in range(epochs): 
        model.train()
        for img0, img1, label in train_all_dataloader:
            img0 = img0.to(device)
            img1 = img1.to(device)
            label = label.to(device)
            emb0 = model(img0)
            emb1 = model(img1)
    
            optimizer.zero_grad()
            loss = criterion(emb0, emb1, label)
            sum_loss += loss.item() * img0.size(0)
            loss_norm += img0.size(0)
            loss.backward()
            optimizer.step()
    
        print(f'Тренировочный лосс: {sum_loss/loss_norm}')
            

In [28]:
train_all(model, train_all_dataloader, criterion, optimizer, 2)

Using 2 GPUs!
Тренировочный лосс: 0.04363622597455978
Тренировочный лосс: 0.0416839426279068


In [30]:
model.load_state_dict(torch.load('/kaggle/working/model_weights_epoch_10.pth'))

<ipython-input-30-6f9cf111d528>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/model_weights_epoch_10.pth'))


<All keys matched successfully>

In [31]:
train_all(model, train_all_dataloader, criterion, optimizer, 2)

Using 2 GPUs!
Тренировочный лосс: 0.0497618784070015
Тренировочный лосс: 0.04635836049616337


In [32]:
torch.save(model.state_dict(), 'model_final_weights_2.0.pth')

Оценка модели 

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics import roc_curve

# Определение функции для вычисления EER
def compute_eer(y_true, y_score):
    fpr, tpr, threshold = roc_curve(y_true, y_score)

    eps = 1e-3
    threshold[0] = max(threshold[1:]) + eps

    fnr = 1 - tpr
    eer_index = np.nanargmin(np.absolute((fnr - fpr)))
    eer = fnr[eer_index]
    return eer

# Определение функции для вычисления косинусного сходства
def cosine_similarity(embedding1, embedding2):
    return 1 - cosine(embedding1.cpu().numpy(), embedding2.cpu().numpy())

# Перенос модели на GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

y_scores = []
y_true = []

# Создание DataLoader для валидационного набора данных
val_dataset = FaceEshkereDataset(all_pairs[:800], 'test')
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=0)

# Проход по данным и вычисление сходства
for pair in val_dataset:
    # Перенос данных на GPU
    img1 = pair[0].unsqueeze(0).to(device)
    img2 = pair[1].unsqueeze(0).to(device)
    
    # Получение эмбеддингов
    emb1 = model(img1)
    emb2 = model(img2)
    
    # Перенос эмбеддингов на CPU для вычисления косинусного сходства
    emb1 = emb1.squeeze(0).detach().cpu()
    emb2 = emb2.squeeze(0).detach().cpu()
    
    # Вычисление косинусного сходства
    similarity = cosine_similarity(emb1, emb2)
    y_scores.append(similarity)
    
    # Преобразование меток: -1 -> 0, 1 -> 1
    y_true.append(1 if pair[2] == 1 else 0)

# Вычисление EER
eer = compute_eer(y_true, y_scores)
print(f"Equal Error Rate (EER): {eer:.4f}")